In [70]:
from utils import profiler, reader
from typing import List
from collections import defaultdict

In [89]:
datafile = "../data/day5_input.txt"
data = reader.read_from_file(datafile)

def process_data(data: List[List[str]]) -> tuple[List[List[int]]]:
    rules = []
    updates = []
    for line in data:
        if "|" in line:
            rules.append(list(map(int, line.rstrip().split("|"))))
        elif line != '\n':
            updates.append(list(map(int, line.rstrip().split(","))))
    
    return (rules, updates)

rules, updates = process_data(data)

# Part 1

### Overview 

Given the dependencies listed in rules, check to make sure that each update follows the rules.

### Approach

Treat the page dependencies as a graph where the edges are dependencies and the vertices are the pages! The input is an edge list. Try to make a list of the dependencies that we can store in an adjacency list. Then the condition is that no dependencies appear AFTER the given item. This is the same as "the rest of the list has NO elements in the list of dependencies".

In [102]:
def graph(data: List[List]):
    # This takes in an edge list and makes an adjacency list of the dependencies
    d = defaultdict(list)
    for dependency, item in data:
        d[item].append(dependency)
    return d

def scan_update(rules, update) -> bool: #An update is invalid if for any given item, any of the items after are a dependency of that item.
    for i, item in enumerate(update):
        # If there are dependencies for this item
        if item in rules:
            # Check to ensure that no dependency comes after! 
            # If the dependencies and rest of the update share an item, the update is invalid
            dependencies = rules[item]
            for x in update[i:]:
                if x in dependencies:
                    return False
    return True

def middle_number(l: List) -> int:
    return l[len(l)//2]

@profiler.profile
def part1(rules: List[List], updates: List[List]) -> int:
    adj_list = graph(rules)
    result = 0

    for update in updates:
        if scan_update(adj_list, update):
            result += middle_number(update)

    return result

part1(rules, updates)

Calling part1: Memory used 225280 kB; Execution Time: 0.007961084134876728 s


5955

# Part 2

### Overview 

Find the incorrectly ordered updates and reorder them correctly. Then sum up the middle elements of all of these.

### Approach

First we extract the items in updates that do not adhere to the rules. Then ordering them correctly looks like a topological sorting! We can do this with a depth first search where we visit the nodes in prerequisite order using our adjacency list to guide us.


In [153]:
def topo_sort(graph: List[int], rules: dict):
    result = []
    seen = set()
    def visit(n: int) -> None:
        if n in seen or n not in graph:
            return
        dependencies = rules[n]
        for node in dependencies:
            visit(node)
        seen.add(n)
        result.append(n)

    for node in graph:
        visit(node)

    return result

@profiler.profile
def part2(rules, updates) -> int:
    adj_list = graph(rules)
    result = 0
    
    for update in updates:
        if not scan_update(adj_list, update):
        
            sorted_list = topo_sort(update, adj_list)
            result += middle_number(sorted_list)


    return(result)


part2(rules, updates)


Calling part2: Memory used 73728 kB; Execution Time: 0.02088716672733426 s


4030